In [22]:
import os
import pandas as pd
import numpy as np
import json
import io

In [23]:
aggregated_results_path = os.path.join(os.getcwd(), "results", "aggregated.csv")
aggregated_results_df = pd.read_csv(aggregated_results_path)

In [24]:
rsa = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "rsa") &
        (aggregated_results_df.cipher_type == "3072") &
        (aggregated_results_df.number_transactions == 1000) & 
        (aggregated_results_df.used_cache == False) &
        (aggregated_results_df.block_size == 9)
      ]
ecc = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "ecc") &
        (aggregated_results_df.cipher_type == "P-256") &
        (aggregated_results_df.number_transactions == 1000) &
        (aggregated_results_df.used_cache == False) &
        (aggregated_results_df.block_size == 9)
      ]
dilithium = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "dilithium") &
        (aggregated_results_df.cipher_type == "Dilithium3") &
        (aggregated_results_df.number_transactions == 1000) &
        (aggregated_results_df.used_cache == False) &
        (aggregated_results_df.block_size == 9)
      ]

rsaCache = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "rsa") &
        (aggregated_results_df.cipher_type == "3072") &
        (aggregated_results_df.number_transactions == 1000) & 
        (aggregated_results_df.used_cache == True) &
        (aggregated_results_df.block_size == 9)
      ]
eccCache = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "ecc") &
        (aggregated_results_df.cipher_type == "P-256") &
        (aggregated_results_df.number_transactions == 1000) &
        (aggregated_results_df.used_cache == True) &
        (aggregated_results_df.block_size == 9)
      ]
dilithiumCache = aggregated_results_df.loc[
        (aggregated_results_df.cipher == "dilithium") &
        (aggregated_results_df.cipher_type == "Dilithium3") &
        (aggregated_results_df.number_transactions == 1000) &
        (aggregated_results_df.used_cache == True) &
        (aggregated_results_df.block_size == 9)
      ]

In [25]:
def getList(input):
    output = pd.read_csv(io.StringIO('\n'.join(input.split('\n')[:-1])), header=None, index_col=0, sep='\s+')
    return output[1].values

In [26]:
rsa_dist = getList(rsa["distribution"].iloc[0])
ecc_dist = getList(ecc["distribution"].iloc[0])
dilithium_dist = getList(dilithium["distribution"].iloc[0])

rsa_cache_dist = getList(rsaCache["distribution"].iloc[0])
ecc_cache_dist = getList(eccCache["distribution"].iloc[0])
dilithium_cache_dist = getList(dilithiumCache["distribution"].iloc[0])

In [27]:
from scipy.stats import ttest_ind, bootstrap

In [28]:
rsa_bt = np.random.choice(rsa_dist, size=9999)
ecc_bt = np.random.choice(ecc_dist, size=9999)
dilithium_bt = np.random.choice(dilithium_dist, size=9999)

In [29]:
print("1000 tx: RSA vs ECC (H: Both equal)", ttest_ind(rsa_dist, ecc_dist, random_state=42))

1000 tx: RSA vs ECC (H: Both equal) Ttest_indResult(statistic=9.785457651389853, pvalue=6.216416350344414e-12)


In [39]:
print("1000 tx: RSA vs ECC (H: Both equal)", ttest_ind(rsa_dist, ecc_dist, random_state=42, permutations=10000))

1000 tx: RSA vs ECC (H: Both equal) Ttest_indResult(statistic=9.785457651389853, pvalue=9.999000099990002e-05)


In [30]:
import random
rsa_shuffle = rsa_dist.copy()
ecc_shuffle = ecc_dist.copy()
random.shuffle(rsa_shuffle)
random.shuffle(ecc_shuffle)

print("1000 tx: RSA vs ECC (H: Both equal)", ttest_ind(rsa_shuffle, ecc_shuffle, random_state=42))

1000 tx: RSA vs ECC (H: Both equal) Ttest_indResult(statistic=9.785457651389853, pvalue=6.216416350344414e-12)


In [46]:
6.216416350344414e-12 * 3

1.8649249051033242e-11

In [31]:
print("1000 tx: RSA vs ECC (H: RSA less)", ttest_ind(rsa_dist, ecc_dist, random_state=42, alternative='less'))

1000 tx: RSA vs ECC (H: RSA less) Ttest_indResult(statistic=9.785457651389853, pvalue=0.9999999999968918)


In [33]:
print("1000 tx: RSA vs Dilithium (H: Both equal)", ttest_ind(rsa_dist, dilithium_dist, random_state=42))

1000 tx: RSA vs Dilithium (H: Both equal) Ttest_indResult(statistic=-21.125584509118795, pvalue=1.3331960668653799e-22)


In [47]:
1.3331960668653799e-22 * 3

3.99958820059614e-22

In [34]:
print("1000 tx: ECC vs Dilithium (H: Both equal)", ttest_ind(ecc_dist, dilithium_dist, random_state=42))

1000 tx: ECC vs Dilithium (H: Both equal) Ttest_indResult(statistic=-24.88165349706574, pvalue=4.0030858503509063e-25)


In [48]:
4.0030858503509063e-25 * 3

1.200925755105272e-24

In [35]:
# cache tests

In [37]:
print("1000 tx: D Cache vs D (H: D Cache Less)", ttest_ind(dilithium_cache_dist, dilithium_dist, random_state=42, alternative='less'))

1000 tx: D Cache vs D (H: D Cache Less) Ttest_indResult(statistic=0.6329465872222688, pvalue=0.734047828734417)


In [44]:
print("1000 tx: ECC Cache vs ECC (H: ECC Cache Less)", ttest_ind(ecc_cache_dist, ecc_dist, random_state=42, alternative='less'))

1000 tx: ECC Cache vs ECC (H: ECC Cache Less) Ttest_indResult(statistic=-0.7000006494577001, pvalue=0.24485127620152003)


In [45]:
print("1000 tx: RSA Cache vs RSA (H: RSA Cache Less)", ttest_ind(rsa_cache_dist, rsa_dist, random_state=42, alternative='less'))

1000 tx: RSA Cache vs RSA (H: RSA Cache Less) Ttest_indResult(statistic=-2.2913122912030603, pvalue=0.01483782119042054)
